# Selected Dataset Construction

This notebook builds the final analysis dataset based on learnings from the data audit.

## Key Insight
**The Twelve dataset only contains cross-league transfers** (e.g., Liga MX → MLS, not América → Santos).
There are ZERO intra-league transfers in this dataset.

## Selection Criteria
1. **Real transfers only** - Different team (excludes promotion/relegation)
2. **Same position** - Comparable before/after metrics
3. **Enriched with all available context** - Names, IDs, market values, etc.

## Analysis Focus
Since we can't filter by "same league", we'll analyze:
- Transfers **FROM** specific leagues (where do Liga MX players go?)
- Transfers **TO** specific leagues (who comes to Premier League?)
- Specific league deep-dives: Mexico, England, Sweden

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 200)

BASE_PATH = "../../thesis_data/"

# Load all datasets
print("Loading datasets...")
df_raw = pd.read_parquet(f"{BASE_PATH}raw_data_twelve/Twelve/male_transfer_model.parquet")
df_comps = pd.read_parquet(f"{BASE_PATH}raw_data_twelve/Wyscout/competitions_wyscout.parquet")
df_players = pd.read_parquet(f"{BASE_PATH}raw_data_twelve/Wyscout/players_wyscout.parquet")
df_mapping = pd.read_parquet(f"{BASE_PATH}raw_data_twelve/Transfermarkt/wy_tm_players_mapping.parquet")
df_tm_history = pd.read_parquet(f"{BASE_PATH}tm_data/transfer_history_all.parquet")

print(f"Raw transfers: {len(df_raw):,} rows")
print("All datasets loaded!")

Loading datasets...
Raw transfers: 78,191 rows
All datasets loaded!


---
## 1. Apply Base Filters

In [2]:
# Filter 1: Real transfers (different team)
real_transfer = df_raw['from_team_id'] != df_raw['to_team_id']
print(f"After real transfers filter: {real_transfer.sum():,} rows")

# Filter 2: Same position
same_pos = df_raw['from_position'] == df_raw['to_position']
print(f"After same position filter: {(real_transfer & same_pos).sum():,} rows")

# Apply filters
df = df_raw[real_transfer & same_pos].copy()
print(f"\nFiltered dataset: {len(df):,} rows")
print(f"Unique players: {df['player_id'].nunique():,}")

After real transfers filter: 56,697 rows
After same position filter: 45,415 rows

Filtered dataset: 45,415 rows
Unique players: 31,983


---
## 2. Enrich with Competition Context

In [3]:
# Get unique competition info
comp_info = df_comps[['competition_id', 'name', 'country', 'division']].drop_duplicates()

# Add FROM competition context
df = df.merge(
    comp_info.rename(columns={
        'competition_id': 'from_competition',
        'name': 'from_league_name',
        'country': 'from_country',
        'division': 'from_division'
    }),
    on='from_competition',
    how='left'
)

# Add TO competition context
df = df.merge(
    comp_info.rename(columns={
        'competition_id': 'to_competition',
        'name': 'to_league_name',
        'country': 'to_country',
        'division': 'to_division'
    }),
    on='to_competition',
    how='left'
)

print(f"Rows after competition enrichment: {len(df):,}")
print(f"\nFrom country coverage: {df['from_country'].notna().mean()*100:.1f}%")
print(f"To country coverage: {df['to_country'].notna().mean()*100:.1f}%")

Rows after competition enrichment: 45,415

From country coverage: 66.1%
To country coverage: 69.0%


---
## 3. Enrich with Player Metadata (Wyscout)

In [4]:
# Select relevant player columns
player_cols = ['player_id', 'name', 'birth_date', 'height', 'weight', 'foot', 
               'passport', 'birth_country', 'role']

# Check for conflicts
existing_cols = set(df.columns)
conflicts = [c for c in player_cols if c in existing_cols and c != 'player_id']
print(f"Columns that already exist: {conflicts}")

# Merge with suffix for conflicts
df = df.merge(
    df_players[player_cols],
    on='player_id',
    how='left',
    suffixes=('', '_wy')
)

# Rename for clarity
rename_map = {
    'name': 'player_name_wy',
    'birth_date_wy': 'birth_date_wy',
    'height': 'height_cm',
    'weight': 'weight_kg',
    'role': 'wy_role'
}
for old, new in rename_map.items():
    if old in df.columns and old != new:
        df = df.rename(columns={old: new})

print(f"\nWyscout player coverage:")
print(f"  player_name_wy: {df['player_name_wy'].notna().mean()*100:.1f}%")
print(f"  height_cm (>0): {(df['height_cm'] > 0).mean()*100:.1f}%")
print(f"  foot: {df['foot'].notna().mean()*100:.1f}%")

Columns that already exist: ['birth_date']

Wyscout player coverage:
  player_name_wy: 20.6%
  height_cm (>0): 20.5%
  foot: 20.4%


---
## 4. Add Transfermarkt ID Mapping

In [5]:
# Add TM ID via mapping
df = df.merge(
    df_mapping.rename(columns={'wy_id': 'player_id'}),
    on='player_id',
    how='left'
)

print(f"TM ID coverage: {df['tm_id'].notna().mean()*100:.1f}%")
print(f"Players with TM ID: {df[df['tm_id'].notna()]['player_id'].nunique():,}")

TM ID coverage: 86.5%
Players with TM ID: 26,845


---
## 5. Add Transfer Fee and Market Value from TM History

This is tricky because we need to match transfers between datasets.

In [6]:
# Check what keys we can use to match
print("=== TM History columns ===")
print(df_tm_history.columns.tolist())

print("\n=== Potential match keys ===")
print(f"TM History has wy_player_id: {df_tm_history['wy_player_id'].notna().mean()*100:.1f}% coverage")
print(f"Our dataset has player_id (which is wy_id)")

=== TM History columns ===
['wy_player_id', 'player_name', 'player_first_name', 'player_last_name', 'player_short_name', 'player_id', 'team_id_from', 'team_name_from', 'team_id_to', 'team_name_to', 'competition_id_from', 'competition_name_from', 'competition_country_from', 'competition_id_to', 'competition_name_to', 'competition_country_to', 'age_at_transfer', 'transfer_fee', 'transfer_value', 'date', 'remaining_contract_period', 'contract_until_date']

=== Potential match keys ===
TM History has wy_player_id: 100.0% coverage
Our dataset has player_id (which is wy_id)


In [7]:
# Prepare TM history for matching
# We'll match on: player_id + approximate season

# Parse date in TM history
df_tm_history['date'] = pd.to_datetime(df_tm_history['date'], errors='coerce')
df_tm_history['tm_year'] = df_tm_history['date'].dt.year

# Aggregate TM history by player + year (take latest transfer that year)
tm_agg = df_tm_history.groupby(['wy_player_id', 'tm_year']).agg({
    'transfer_fee': 'last',
    'transfer_value': 'last',
    'age_at_transfer': 'last',
    'team_name_from': 'last',
    'team_name_to': 'last',
    'competition_id_from': 'last',
    'competition_id_to': 'last'
}).reset_index()

tm_agg = tm_agg.rename(columns={
    'wy_player_id': 'player_id',
    'transfer_fee': 'tm_transfer_fee',
    'transfer_value': 'tm_market_value',
    'age_at_transfer': 'tm_age_at_transfer',
    'team_name_from': 'tm_team_from',
    'team_name_to': 'tm_team_to',
    'competition_id_from': 'tm_comp_from',
    'competition_id_to': 'tm_comp_to'
})

print(f"TM aggregated records: {len(tm_agg):,}")

TM aggregated records: 259,455


In [8]:
# Create year column for matching
df['transfer_year'] = df['to_season']  # Assuming to_season is the year of transfer

# Join with TM data
df = df.merge(
    tm_agg,
    left_on=['player_id', 'transfer_year'],
    right_on=['player_id', 'tm_year'],
    how='left'
)

print(f"\nTM data coverage after join:")
print(f"  tm_transfer_fee: {df['tm_transfer_fee'].notna().mean()*100:.1f}%")
print(f"  tm_market_value: {df['tm_market_value'].notna().mean()*100:.1f}%")
print(f"  tm_market_value > 0: {(df['tm_market_value'] > 0).mean()*100:.1f}%")


TM data coverage after join:
  tm_transfer_fee: 51.2%
  tm_market_value: 77.5%
  tm_market_value > 0: 65.0%


---
## 6. Create Final Clean Dataset

In [9]:
# Define column groups
id_cols = ['player_id', 'tm_id', 'short_name', 'player_name_wy']

transfer_context = [
    'from_team_id', 'to_team_id',
    'from_competition', 'to_competition',
    'from_league_name', 'to_league_name',
    'from_country', 'to_country',
    'from_division', 'to_division',
    'from_season', 'to_season',
    'from_position', 'to_position'
]

player_meta = [
    'birth_date', 'height_cm', 'weight_kg', 'foot', 
    'passport', 'birth_country'
]

tm_data = [
    'tm_transfer_fee', 'tm_market_value', 'tm_age_at_transfer',
    'tm_team_from', 'tm_team_to'
]

# Performance metrics (z-scores)
z_score_cols = [c for c in df.columns if 'z_score' in c]
minutes_cols = ['from_Minutes', 'to_Minutes']

# Select available columns
all_desired = id_cols + transfer_context + player_meta + tm_data + minutes_cols + z_score_cols
available = [c for c in all_desired if c in df.columns]
missing = [c for c in all_desired if c not in df.columns]

print(f"Available columns: {len(available)}")
print(f"Missing columns: {missing}")

# Create final dataset
df_final = df[available].copy()

Available columns: 181
Missing columns: []


In [10]:
print("=" * 70)
print("FINAL DATASET SUMMARY")
print("=" * 70)

print(f"\nRows: {len(df_final):,}")
print(f"Columns: {len(df_final.columns)}")
print(f"Unique players: {df_final['player_id'].nunique():,}")
print(f"Unique source leagues: {df_final['from_league_name'].nunique()}")
print(f"Unique destination leagues: {df_final['to_league_name'].nunique()}")

print(f"\nSeason range: {df_final['from_season'].min()} - {df_final['from_season'].max()}")

print(f"\nData coverage:")
print(f"  TM ID: {df_final['tm_id'].notna().mean()*100:.1f}%")
print(f"  Market value (>0): {(df_final['tm_market_value'] > 0).mean()*100:.1f}%")
print(f"  Transfer fee (>0): {(df_final['tm_transfer_fee'] > 0).mean()*100:.1f}%")
print(f"  From country: {df_final['from_country'].notna().mean()*100:.1f}%")

FINAL DATASET SUMMARY

Rows: 45,415
Columns: 181
Unique players: 31,983
Unique source leagues: 151
Unique destination leagues: 149

Season range: 2018 - 2025

Data coverage:
  TM ID: 86.5%
  Market value (>0): 65.0%
  Transfer fee (>0): 9.5%
  From country: 66.1%


---
## 7. League Analysis: Where is the Most Data?

In [11]:
# Analyze by source league
league_stats = df_final.groupby(['from_country', 'from_league_name']).agg(
    n_rows=('player_id', 'size'),
    n_players=('player_id', 'nunique'),
    pct_with_market_value=('tm_market_value', lambda x: (x > 0).mean() * 100),
    avg_market_value=('tm_market_value', lambda x: x[x > 0].mean() if (x > 0).any() else 0)
).reset_index()

league_stats = league_stats.sort_values('n_rows', ascending=False)

print("=== TOP 25 SOURCE LEAGUES ===")
print(league_stats.head(25).to_string())

=== TOP 25 SOURCE LEAGUES ===
    from_country            from_league_name  n_rows  n_players  pct_with_market_value  avg_market_value
45        Brazil                     Serie B     692        572              75.433526      4.350575e+05
5      Argentina            Primera Nacional     602        569              73.421927      2.205204e+05
161        Spain            Segunda División     582        536              95.876289      9.944892e+05
4      Argentina  Liga Profesional de Fútbol     536        506              80.783582      1.949423e+06
141     Portugal               Primeira Liga     522        488              89.655172      3.330021e+06
170       Turkey                   Süper Lig     489        462              92.229039      1.890576e+06
72       England                Championship     476        453              89.495798      3.163556e+06
102        Italy                     Serie B     460        428              91.086957      9.205251e+05
44        Brazil         

In [12]:
# Leagues with good data coverage (100+ rows AND >50% market value coverage)
good_leagues = league_stats[
    (league_stats['n_rows'] >= 100) & 
    (league_stats['pct_with_market_value'] >= 50)
].sort_values('n_rows', ascending=False)

print(f"=== LEAGUES WITH GOOD DATA ({len(good_leagues)} leagues) ===")
print("Criteria: 100+ rows AND >50% market value coverage\n")
print(good_leagues.to_string())

=== LEAGUES WITH GOOD DATA (98 leagues) ===
Criteria: 100+ rows AND >50% market value coverage

             from_country            from_league_name  n_rows  n_players  pct_with_market_value  avg_market_value
45                 Brazil                     Serie B     692        572              75.433526      4.350575e+05
5               Argentina            Primera Nacional     602        569              73.421927      2.205204e+05
161                 Spain            Segunda División     582        536              95.876289      9.944892e+05
4               Argentina  Liga Profesional de Fútbol     536        506              80.783582      1.949423e+06
141              Portugal               Primeira Liga     522        488              89.655172      3.330021e+06
170                Turkey                   Süper Lig     489        462              92.229039      1.890576e+06
72                England                Championship     476        453              89.495798      3.163

---
## 8. Deep Dive: Mexico

In [13]:
df_mexico = df_final[df_final['from_country'] == 'Mexico'].copy()

print("=== MEXICO: TRANSFERS OUT ===")
print(f"Total rows: {len(df_mexico):,}")
print(f"Unique players: {df_mexico['player_id'].nunique()}")

print(f"\nSource leagues in Mexico:")
print(df_mexico['from_league_name'].value_counts())

print(f"\nWhere do they go?")
print(df_mexico['to_country'].value_counts().head(10))

print(f"\nMarket value coverage: {(df_mexico['tm_market_value'] > 0).mean()*100:.1f}%")

if (df_mexico['tm_market_value'] > 0).any():
    print(f"\nMarket value stats (when available):")
    print(df_mexico[df_mexico['tm_market_value'] > 0]['tm_market_value'].describe())

=== MEXICO: TRANSFERS OUT ===
Total rows: 416
Unique players: 386

Source leagues in Mexico:
from_league_name
Liga MX                 211
Liga de Expansión MX    205
Name: count, dtype: int64

Where do they go?
to_country
Mexico           75
Argentina        41
United States    33
Uruguay          23
Colombia         16
Chile            16
Guatemala        15
Brazil           14
Costa Rica       14
Spain            13
Name: count, dtype: int64

Market value coverage: 86.1%

Market value stats (when available):
count    3.580000e+02
mean     1.190964e+06
std      1.459336e+06
min      4.000000e+04
25%      4.000000e+05
50%      6.875000e+05
75%      1.500000e+06
max      1.000000e+07
Name: tm_market_value, dtype: float64


In [14]:
# Players coming TO Mexico
df_to_mexico = df_final[df_final['to_country'] == 'Mexico'].copy()

print("=== MEXICO: TRANSFERS IN ===")
print(f"Total rows: {len(df_to_mexico):,}")
print(f"Unique players: {df_to_mexico['player_id'].nunique()}")

print(f"\nWhere do they come from?")
print(df_to_mexico['from_country'].value_counts().head(10))

=== MEXICO: TRANSFERS IN ===
Total rows: 554
Unique players: 506

Where do they come from?
from_country
Mexico           75
Argentina        53
Colombia         27
United States    26
Chile            23
Spain            20
Ecuador          18
Uruguay          15
Paraguay         13
Brazil           10
Name: count, dtype: int64


---
## 9. Deep Dive: England

In [15]:
df_england = df_final[df_final['from_country'] == 'England'].copy()

print("=== ENGLAND: TRANSFERS OUT ===")
print(f"Total rows: {len(df_england):,}")
print(f"Unique players: {df_england['player_id'].nunique()}")

print(f"\nSource leagues in England:")
print(df_england['from_league_name'].value_counts())

print(f"\nWhere do they go?")
print(df_england['to_country'].value_counts().head(10))

print(f"\nMarket value coverage: {(df_england['tm_market_value'] > 0).mean()*100:.1f}%")

# Premier League specific
df_pl = df_england[df_england['from_league_name'] == 'Premier League']
print(f"\n--- Premier League only ---")
print(f"Rows: {len(df_pl)}")
print(f"Where do PL players go?")
print(df_pl['to_country'].value_counts().head(10))

=== ENGLAND: TRANSFERS OUT ===
Total rows: 761
Unique players: 702

Source leagues in England:
from_league_name
Championship      476
Premier League    285
Name: count, dtype: int64

Where do they go?
to_country
England          124
France            62
Spain             56
Turkey            52
Italy             49
Germany           45
Scotland          29
United States     22
Belgium           19
Saudi Arabia      18
Name: count, dtype: int64

Market value coverage: 90.0%

--- Premier League only ---
Rows: 285
Where do PL players go?
to_country
England         45
Spain           43
Italy           39
France          33
Turkey          27
Germany         26
Saudi Arabia    15
Portugal         8
Brazil           7
Qatar            6
Name: count, dtype: int64


In [16]:
# Players coming TO England
df_to_england = df_final[df_final['to_country'] == 'England'].copy()

print("=== ENGLAND: TRANSFERS IN ===")
print(f"Total rows: {len(df_to_england):,}")
print(f"Unique players: {df_to_england['player_id'].nunique()}")

print(f"\nWhere do they come from?")
print(df_to_england['from_country'].value_counts().head(10))

print(f"\nWhich English leagues receive players?")
print(df_to_england['to_league_name'].value_counts())

=== ENGLAND: TRANSFERS IN ===
Total rows: 861
Unique players: 786

Where do they come from?
from_country
England        124
France          83
Germany         60
Italy           58
Spain           57
Belgium         41
Scotland        40
Netherlands     37
Turkey          26
Portugal        21
Name: count, dtype: int64

Which English leagues receive players?
to_league_name
Championship      486
Premier League    375
Name: count, dtype: int64


---
## 10. Deep Dive: Sweden

In [17]:
df_sweden = df_final[df_final['from_country'] == 'Sweden'].copy()

print("=== SWEDEN: TRANSFERS OUT ===")
print(f"Total rows: {len(df_sweden):,}")
print(f"Unique players: {df_sweden['player_id'].nunique()}")

print(f"\nSource leagues in Sweden:")
print(df_sweden['from_league_name'].value_counts())

print(f"\nWhere do they go?")
print(df_sweden['to_country'].value_counts().head(10))

print(f"\nMarket value coverage: {(df_sweden['tm_market_value'] > 0).mean()*100:.1f}%")

=== SWEDEN: TRANSFERS OUT ===
Total rows: 587
Unique players: 522

Source leagues in Sweden:
from_league_name
Allsvenskan    319
Superettan     268
Name: count, dtype: int64

Where do they go?
to_country
Sweden           119
Norway            46
Denmark           30
United States     22
Poland            20
Netherlands       16
Belgium           16
Greece            15
Finland           15
Iceland           13
Name: count, dtype: int64

Market value coverage: 79.2%


In [18]:
# Players coming TO Sweden
df_to_sweden = df_final[df_final['to_country'] == 'Sweden'].copy()

print("=== SWEDEN: TRANSFERS IN ===")
print(f"Total rows: {len(df_to_sweden):,}")
print(f"Unique players: {df_to_sweden['player_id'].nunique()}")

print(f"\nWhere do they come from?")
print(df_to_sweden['from_country'].value_counts().head(10))

=== SWEDEN: TRANSFERS IN ===
Total rows: 562
Unique players: 521

Where do they come from?
from_country
Sweden           119
Norway            66
Denmark           37
Finland           20
United States     16
Netherlands       13
Iceland           11
Cyprus             8
Turkey             8
Greece             6
Name: count, dtype: int64


---
## 11. Save Datasets

In [19]:
# Save main dataset
output_path = "../../thesis_data/processed/"

# Create directory if needed
import os
os.makedirs(output_path, exist_ok=True)

# Save full dataset
df_final.to_parquet(f"{output_path}transfers_clean.parquet", index=False)
print(f"Saved: transfers_clean.parquet ({len(df_final):,} rows)")

# Save country-specific datasets
df_mexico_full = df_final[(df_final['from_country'] == 'Mexico') | (df_final['to_country'] == 'Mexico')]
df_mexico_full.to_parquet(f"{output_path}transfers_mexico.parquet", index=False)
print(f"Saved: transfers_mexico.parquet ({len(df_mexico_full):,} rows)")

df_england_full = df_final[(df_final['from_country'] == 'England') | (df_final['to_country'] == 'England')]
df_england_full.to_parquet(f"{output_path}transfers_england.parquet", index=False)
print(f"Saved: transfers_england.parquet ({len(df_england_full):,} rows)")

df_sweden_full = df_final[(df_final['from_country'] == 'Sweden') | (df_final['to_country'] == 'Sweden')]
df_sweden_full.to_parquet(f"{output_path}transfers_sweden.parquet", index=False)
print(f"Saved: transfers_sweden.parquet ({len(df_sweden_full):,} rows)")

Saved: transfers_clean.parquet (45,415 rows)
Saved: transfers_mexico.parquet (895 rows)
Saved: transfers_england.parquet (1,498 rows)
Saved: transfers_sweden.parquet (1,030 rows)


In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════════╗
║                         DATASET READY                                    ║
╠══════════════════════════════════════════════════════════════════════════╣
║                                                                          ║
║  IMPORTANT NOTE:                                                         ║
║  This dataset contains CROSS-LEAGUE transfers only.                      ║
║  There are NO intra-league transfers (e.g., América → Santos).           ║
║                                                                          ║
║  WHAT YOU CAN ANALYZE:                                                   ║
║  - How do Liga MX players perform after leaving Mexico?                  ║
║  - What's the success rate of transfers TO the Premier League?           ║
║  - Do Swedish players improve when moving to bigger leagues?             ║
║                                                                          ║
║  WHAT YOU CANNOT ANALYZE (with this data):                               ║
║  - Intra-league transfers (within same competition)                      ║
║  - Local market dynamics                                                 ║
║                                                                          ║
║  FILES SAVED:                                                            ║
║  - thesis_data/processed/transfers_clean.parquet (all data)              ║
║  - thesis_data/processed/transfers_mexico.parquet                        ║
║  - thesis_data/processed/transfers_england.parquet                       ║
║  - thesis_data/processed/transfers_sweden.parquet                        ║
║                                                                          ║
╚══════════════════════════════════════════════════════════════════════════╝
""")